In [1]:
# Import all the things

import json
import pandas as pd
import numpy as np
import datetime
import acquire
from prepare import basic_clean, tokenize, lemmatize, stem, remove_stopwords
import prepare
import wrangle 
import re
from sklearn.feature_extraction.text import TfidfVectorizer

from wordcloud import WordCloud
from matplotlib import pyplot as plt
import seaborn as sns

import nltk

from scipy import stats

import warnings
warnings.filterwarnings("ignore")

## Initial Planning Analysis

In [10]:
# # read in primary utterances csv for dataframe
# df = pd.read_csv('./utterances.csv')
# df.head()

,episode,episode_order,speaker,utterance
0,57264,9,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...","It's a 2,200-mile race. To give some sense of ..."
1,57264,10,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",So for a top competitor like Lance to try to m...
2,57264,11,"NEAL CONAN, host","So in every team, presumably there's one star,..."
3,57264,12,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",That's right. Each team has nine riders. And w...
4,57264,13,"NEAL CONAN, host","So slipstream, this is like drafting in car ra..."


In [ ]:
# # find utterance count per episode
# df.episode.value_counts()

19633    569
35108    552
57481    539
73336    492
66523    480
        ... 
80400      1
61102      1
88047      1
73497      1
92554      1
Name: episode, Length: 104920, dtype: int64

In [ ]:
# # find utterance count per episode
# test = df.episode.value_counts() < 2
# test.sum()

1357

In [12]:
# # pull in episodes df
# ep_df = pd.read_csv('./episodes.csv')
# ep_df.head()

In [ ]:
# # check episode content to see if it matches episode id
# df[df.episode == 92554].utterance.to_list()

["Good morning. I'm David Greene. Here's the weather forecast for Albuquerque, New Mexico today - cloudy with a chance of grasshoppers. Yes, that's right. The animals are swarming the city so densely, local radar is mistaking them for rain crowds. These insects are hovering in masses as high as a thousand feet up. Officials say this is the worst infestation in 20 years, but it should pass soon. With any luck, it will go back to raining rain - helping to clean up the mess left behind by these grasshoppers. You're listening to MORNING EDITION."]

In [ ]:
# # check ep_df.id = df.episode
# ep_df[ep_df.id == 92554].title.to_list()

["Albuquerque's Weather? Cloudy With A Chance of Grasshoppers"]

### Initial takeaways
There are episodes with one utterance, showing less interview setting and more reporting.

ep_df.id == df.episode

### Acquire


In [21]:
# df = pd.read_csv('./utterances.csv')
# ep_df = pd.read_csv('./episodes.csv')


In [3]:
# df['speaker'] = df.speaker.str.lower()
# df['program'] = df.program.str.lower()
# df['title'] = df.title.str.lower()

In [23]:
# # joining utterances df ('df') and episodes on 'id'
# joined_df = pd.merge(df, ep_df, left_on = 'episode', right_on='id', how = 'inner')
# joined_df.drop(columns = ['id'], inplace=True)
# joined_df.rename(columns={'episode':'episode_id'}, inplace = True)

In [24]:
# joined_df['is_host'] = joined_df.speaker.str.contains(r'\W*(host)\W*')

In [7]:
# df = acquire.get_df()

,episode_id,episode_order,speaker,utterance,program,title,episode_date,is_host
0,57264,9,"ms. loren mooney (editor-in-chief, bicycling m...","It's a 2,200-mile race. To give some sense of ...",talk of the nation,how to watch the tour de france,2010-07-12,False
1,57264,10,"ms. loren mooney (editor-in-chief, bicycling m...",So for a top competitor like Lance to try to m...,talk of the nation,how to watch the tour de france,2010-07-12,False
2,57264,11,"neal conan, host","So in every team, presumably there's one star,...",talk of the nation,how to watch the tour de france,2010-07-12,True
3,57264,12,"ms. loren mooney (editor-in-chief, bicycling m...",That's right. Each team has nine riders. And w...,talk of the nation,how to watch the tour de france,2010-07-12,False
4,57264,13,"neal conan, host","So slipstream, this is like drafting in car ra...",talk of the nation,how to watch the tour de france,2010-07-12,True


## Preparation

- nulls
- duplicates
- create sentiment score
- use prep functions
- keep some punctuation
- split
- date to datetime

In [4]:

# # obtain top 10 hosts
# hosts_to_keep = df[df.is_host == True].speaker.value_counts().head(10).index.to_list()
# # create host df
# hosts_df = df[df.speaker.isin(hosts_to_keep)]
# # get episode_id of top 10 hosts
# top_host_episodes = hosts_df.episode_id.value_counts().index.to_list()
# # create dataframe with mask of episodes with top hosts
# df = df[df.episode_id.isin(top_host_episodes)]
# # remove rows with foreign languages spoken
# df = df[df.utterance!='(Foreign language spoken)']
# # remove rows without speaker (sound effects)
# df = df[df.speaker!='_NO_SPEAKER']
# # drop duplicates
# df.drop_duplicates(inplace = True)
# # drop nulls
# df.dropna(inplace=True)
# # create clean column
# df['clean'] = [tokenize(basic_clean(u)) for u in df.utterance]
# # create lemmatized column
# df['lemmatized'] = df['clean'].apply(tokenize).apply(lemmatize)
# # vader sentiment analysis
# sia = nltk.sentiment.SentimentIntensityAnalyzer()
# df['vader'] = df.lemmatized.apply(lambda doc: sia.polarity_scores(doc)['compound'])
# # date column to datetime
# df['date'] = pd.to_datetime(df.episode_date)
# # cutoff dates prior to 2005 due to low observation count
# df = df[df.date > '2005']
# # double check drop nulls
# df.dropna(inplace = True)

In [27]:
# # obtain top 10 hosts
# hosts_to_keep = df[df.is_host == True].speaker.value_counts().head(10).index.to_list()
# # create host df
# hosts_df = df[df.speaker.isin(hosts_to_keep)]
# # get episode_id of top 10 hosts
# top_host_episodes = hosts_df.episode_id.value_counts().index.to_list()


In [30]:
# create dataframe with mask of episodes with top hosts
# df = df[df.episode_id.isin(top_host_episodes)]

In [31]:
# # drop nulls
# df.dropna(inplace=True)

In [32]:
# drop duplicates
# df.drop_duplicates(inplace = True)

In [33]:
# # remove rows without speaker (sound effects)
# df = df[df.speaker!='_NO_SPEAKER']

In [34]:
# # remove rows with foreign languages spoken
# df = df[df.utterance!='(Foreign language spoken)']


In [35]:
# # create clean column
# df['clean'] = [tokenize(basic_clean(u)) for u in df.utterance]


In [10]:
# # create lemmatized column
# df['lemmatized'] = df['clean'].apply(tokenize).apply(lemmatize)


In [72]:
# df.head()

,episode_id,episode_order,speaker,utterance,program,title,is_host,clean,lemmatized,vader,date
0,57264,9,"ms. loren mooney (editor-in-chief, bicycling m...","It's a 2,200-mile race. To give some sense of ...",talk of the nation,how to watch the tour de france,False,"it s a 2,200 mile race. to give some sense of ...","it s a 2,200 mile race. to give some sense of ...",0.0000,2010-07-12
1,57264,10,"ms. loren mooney (editor-in-chief, bicycling m...",So for a top competitor like Lance to try to m...,talk of the nation,how to watch the tour de france,False,so for a top competitor like lance to try to m...,so for a top competitor like lance to try to m...,0.9346,2010-07-12
2,57264,11,"neal conan, host","So in every team, presumably there's one star,...",talk of the nation,how to watch the tour de france,True,"so in every team , presumably there s one star...","so in every team , presumably there s one star...",0.7096,2010-07-12
3,57264,12,"ms. loren mooney (editor-in-chief, bicycling m...",That's right. Each team has nine riders. And w...,talk of the nation,how to watch the tour de france,False,that s right. each team has nine riders. and w...,that s right. each team ha nine riders. and wh...,0.9274,2010-07-12
4,57264,13,"neal conan, host","So slipstream, this is like drafting in car ra...",talk of the nation,how to watch the tour de france,True,"so slipstream , this is like drafting in car r...","so slipstream , this is like drafting in car r...",0.3612,2010-07-12


In [2]:
# df = wrangle.get_npr_data()

Getting top hosts...
Getting Episode ID's for the hosts...
Double checking speaker variables...
Dropping duplicates...
Dropping null values...
Cleaning corpus...
Lemmatizing corpus...
Analyzing sentiment with VADER...
Converting to datetime...
Trimming timeline...
The df has 1889857 rows and 12 columns.


## JOSH
- Are there words that are said more frequently by hosts? By time of day? By category?
What host(s) say(s) the most words?

In [2]:
df = wrangle.get_npr_data()

The df has 1889857 rows and 12 columns.


In [101]:
df['lemmatized'] = df.lemmatized.astype(str)

In [102]:
train, validate, test = wrangle.split_data(df)

### Hosts

In [7]:
# df of hosts
host_df = df[df.is_host==True]
# hosts with most interviews
hosts_with_the_most = host_df.speaker.value_counts().head(10).index.to_list()
hosts_with_the_most


['neal conan, host',
 'ira flatow, host',
 'steve inskeep, host',
 'robert siegel, host',
 'melissa block, host',
 'renee montagne, host',
 'farai chideya, host',
 'scott simon, host',
 'rachel martin, host',
 'david greene, host']

In [104]:
# aggregate utterances by the speakers that are in top 10 hosts
host_words = train[train.speaker.isin(hosts_with_the_most)].groupby('speaker')['clean'].agg(lambda col: ' '.join(col))

In [60]:
def string_cleaning(s):
    '''
    Function to remove punctuation from word frequencies for hosts.
    '''
    # remove special characters
    s = re.sub(r"[^a-z0-9'\s]", '', s)

    return s

In [105]:
host_words = host_words.apply(string_cleaning).apply(remove_stopwords)

In [106]:
# create word frequencies for each host
greene_freq = pd.Series(host_words['david greene, host'].split()).value_counts()
martin_freq = pd.Series(host_words['rachel martin, host'].split()).value_counts()
simon_freq = pd.Series(host_words['scott simon, host'].split()).value_counts()
chideya_freq = pd.Series(host_words['farai chideya, host'].split()).value_counts()
montagne_freq = pd.Series(host_words['renee montagne, host'].split()).value_counts()
block_freq = pd.Series(host_words['melissa block, host'].split()).value_counts()
siegel_freq = pd.Series(host_words['robert siegel, host'].split()).value_counts()
inskeep_freq = pd.Series(host_words['steve inskeep, host'].split()).value_counts()
flatow_freq = pd.Series(host_words['ira flatow, host'].split()).value_counts()
conan_freq = pd.Series(host_words['neal conan, host'].split()).value_counts()

In [107]:
word_counts = pd.concat([conan_freq, flatow_freq, inskeep_freq, siegel_freq, block_freq, montagne_freq, chideya_freq, simon_freq, martin_freq, greene_freq], axis=1).fillna(0).astype(int)
word_counts.columns = ['conan', 'flatow', 'inskeep', 'siegel', 'block', 'montagne', 'chideya', 'simon', 'martin', 'greene']
word_counts

,conan,flatow,inskeep,siegel,block,montagne,chideya,simon,martin,greene
us,39584,5614,2824,4716,2673,3294,3200,5038,2991,2054
talk,18267,3861,923,552,800,448,996,419,522,564
much,14374,1225,2217,1425,1340,1435,1282,2542,1498,801
well,14351,2572,1559,2417,1505,1096,2121,905,540,1290
let,13896,3366,1610,622,617,541,1541,682,749,972
...,...,...,...,...,...,...,...,...,...,...
charbroiling,0,0,0,0,0,0,0,0,0,1
ravech,0,0,0,0,0,0,0,0,0,1
majook,0,0,0,0,0,0,0,0,0,1
stewardships,0,0,0,0,0,0,0,0,0,1


In [108]:
# create dictionary for word clouds
word_lists = {'conan':conan_freq, 
'flatow':flatow_freq, 
'inskeep': inskeep_freq, 
'siegel':siegel_freq, 
'block':block_freq, 
'montagne':montagne_freq, 
'chideya':chideya_freq, 
'simon':simon_freq, 
'martin':martin_freq, 
'greene':greene_freq}

In [109]:
# word clouds
for h in word_lists:
    plt.figure(figsize=(7,7))
    img = WordCloud(background_color='white', width=800, height=600).generate(word_lists[h])
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'{h}')

TypeError: expected string or bytes-like object

<Figure size 504x504 with 0 Axes>

In [125]:
train['message_length'] = train.clean.apply(len)
train['word_count'] = train.clean.apply(basic_clean).apply(str.split).apply(len)

validate['message_length'] = validate.clean.apply(len)
validate['word_count'] = validate.clean.apply(basic_clean).apply(str.split).apply(len)

test['message_length'] = test.clean.apply(len)
test['word_count'] = test.clean.apply(basic_clean).apply(str.split).apply(len)

train.head()

TypeError: object of type 'float' has no len()

In [124]:
print(train[train.speaker.isin(hosts_with_the_most)].groupby('speaker').message_length.mean())
print(train[train.speaker.isin(hosts_with_the_most)].groupby('speaker').word_count.mean())
print(train[train.speaker.isin(hosts_with_the_most)].groupby('speaker').word_count.sum())

speaker
david greene, host      178.760259
farai chideya, host     190.173800
ira flatow, host        122.776262
melissa block, host     164.656432
neal conan, host        148.899234
rachel martin, host     157.298332
renee montagne, host    171.041212
robert siegel, host     174.635140
scott simon, host       181.393062
steve inskeep, host     168.979687
Name: message_length, dtype: float64
speaker
david greene, host      34.854750
farai chideya, host     37.695985
ira flatow, host        25.493543
melissa block, host     31.889422
neal conan, host        30.021699
rachel martin, host     30.763272
renee montagne, host    32.494347
robert siegel, host     33.524589
scott simon, host       35.161734
steve inskeep, host     32.412296
Name: word_count, dtype: float64
speaker
david greene, host       555515
farai chideya, host      768018
ira flatow, host        1044292
melissa block, host      711453
neal conan, host        4293223
rachel martin, host      523837
renee montagne, host    

In [122]:
train[train.speaker.isin(hosts_with_the_most)].groupby(['speaker','episode_id']).message_length.sum()

speaker              episode_id
david greene, host   777           1742
                     780            189
                     800            667
                     802            708
                     804            489
                                   ... 
steve inskeep, host  133180          18
                     133182          23
                     133184         453
                     135547         365
                     138265        1624
Name: message_length, Length: 56725, dtype: int64

In [123]:
train[train.speaker.isin(hosts_with_the_most)].groupby(['speaker','episode_id']).word_count.sum()

speaker              episode_id
david greene, host   777           335
                     780            38
                     800           121
                     802           141
                     804            91
                                  ... 
steve inskeep, host  133180          5
                     133182          6
                     133184         80
                     135547         69
                     138265        311
Name: word_count, Length: 56725, dtype: int64